In [95]:
user_story_id = 114352

In [96]:
# Obtained from main.py
from jira import JIRA
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication

def get_azure_devops_connection():
    personal_access_token = AZURE_PAT_TOKEN
    organization_url = ADO_ORG_URL
    credentials = BasicAuthentication('', personal_access_token)
    connection = Connection(base_url=organization_url, creds=credentials)
    return connection

def get_jira_connection():
    options = {"server": JIRA_ORG_URL}
    API_KEY = JIRA_API_KEY
    user = MY_WORK_EMAIL
    jira = JIRA(options, basic_auth=(user, API_KEY))
    return jira

def get_work_item(user_story_id, connection):
    client = connection.clients.get_work_item_tracking_client()
    work_item = client.get_work_item(user_story_id, expand="Relations")
    return work_item

def get_work_item_title(work_item):
    return work_item.fields['System.Title']

def get_user_story_children_ids(work_item):

    child_ids = [
        x.url.split("/")[-1] 
        for x in work_item.relations 
            if x.attributes['name'] == "Child" 
    ]
    
    return child_ids

In [97]:
jira_conn = get_jira_connection()
ado_conn = get_azure_devops_connection()

In [98]:
import plotly.express as px
import pandas as pd
from datetime import datetime
from datetime import timedelta

TITLE = "System.Title"
STORY_POINTS = "Microsoft.VSTS.Scheduling.StoryPoints"
START = "Microsoft.VSTS.Scheduling.TargetDate"
ado_format = "%Y-%m-%dT%H:%M:%SZ"
new_format = "%Y-%m-%d"

work_item = get_work_item(user_story_id, ado_conn)
work_item_name = get_work_item_title(work_item)
child_ids = sorted(get_user_story_children_ids(work_item))

gant_chart_data = []

tasks = []
for child_id in child_ids:
    work_item = get_work_item(child_id, ado_conn)

    ado_start_date = work_item.fields.get(START)
    if ado_start_date is None:
        continue

    start_datetime_obj = datetime.strptime(ado_start_date, ado_format)
    new_start_date = start_datetime_obj.strftime(new_format)
    finish_datetime_obj = start_datetime_obj + timedelta(days=work_item.fields[STORY_POINTS])
    new_finish_date = finish_datetime_obj.strftime(new_format)

    x = {
        "Task": work_item.fields[TITLE],
        #"estimate": work_item.fields[STORY_POINTS],
        "Start": new_start_date,
        "Finish": new_finish_date
    }
    print(x)
    gant_chart_data.append(x)

    



{'Task': 'Documentation (release notes, etc)', 'Start': '2021-02-17', 'Finish': '2021-02-19'}
{'Task': 'Add switch to OPC to call the lite camera', 'Start': '2021-02-08', 'Finish': '2021-02-10'}
{'Task': 'Define test cases + configs for OOXML (DOCX, XLSX, PPTX)', 'Start': '2021-02-25', 'Finish': '2021-02-28'}
{'Task': 'Test execution and result verification', 'Start': '2021-03-02', 'Finish': '2021-03-05'}
{'Task': 'Create BDDL and DVL definitions', 'Start': '2021-02-01', 'Finish': '2021-02-06'}
{'Task': 'Autogenerate and integrate generated code and Apply necessary modifications', 'Start': '2021-02-08', 'Finish': '2021-02-16'}
{'Task': 'Create test sets', 'Start': '2021-02-19', 'Finish': '2021-02-22'}
{'Task': 'Test BDD definitions (Miraplacid, bin directory reader)', 'Start': '2021-01-29', 'Finish': '2021-01-31'}


# Target Timeline

In [102]:

df = pd.DataFrame(gant_chart_data)

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", title=f"Target Timeline of User Story {user_story_id}: {work_item_name}")
#fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.update_layout(autosize=False, width=2000, height=400)
fig.show()